In [1]:
#安装相关依赖库 如果是windows系统，cmd命令框中输入pip安装，或在Jupyter notebook中!pip安装，参考上述环境配置
#!pip install pandas numpy cv2 torch torchvision time albumentations timm tqdm

#---------------------------------------------------
#导入库
from PIL import Image
import cv2
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torchvision import models


import timm
import albumentations as A

C:\Users\BiXY\anaconda3\envs\mmcv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#----------------框架设置----------------
#设置torch使用gpu
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#输出cuda说明使用gpu，输出cpu说明使用cpu，最好使用gpu训练
print(device)

cuda


In [3]:
#----------------数据处理----------------
#基础数据读取
train_df = pd.read_csv('data/train.csv')
train_df['path'] = 'data/train/' + train_df['image']


test_df = pd.read_csv('data/提交示例.csv')
test_df['path'] = 'data/test/' + test_df['image'] 

class XunFeiDataset(Dataset):
    def __init__(self, img_path, label, transforms=None):        
        self.img_path = img_path        
        self.label = label        
        if transforms is not None:           
            self.transform = transforms        
        else:            
            self.transform = None        
    def __getitem__(self, index):        
        img = cv2.imread(self.img_path[index])                    
        img = img.astype(np.float32)                
        img /= 255.0        
        img -= 1                
        
        if self.transform is not None:            
            img = self.transform(image = img)['image']        
        img = img.transpose([2,0,1])                
        
        return img,torch.from_numpy(np.array(self.label[index]))
    
    def __len__(self):        
        return len(self.img_path)



In [ ]:
import torchvision.transforms as transforms       
#----------------模型训练----------------
       
# 模型训练一个epoch的函数
def train(train_loader, model, criterion, optimizer):   
    model.train()  
    train_acc = 0.0  
    train_loss = 0.0    
    
    for i, (input, target) in enumerate(train_loader):               
        input = input.to(device)        
        target = target.to(device)        
        output = model(input)      
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()        
        optimizer.step()
        if i % 40 == 0:            
            print(loss.item())
        train_acc += (output.argmax(1) == target).sum().item()                             
        train_loss += loss.item()        
    
    return train_loss/len(train_loader.dataset),train_acc / len(train_loader.dataset)

# 模型验证一个epoch的函数
# 模型验证一个epoch的函数
def validate(val_loader, model, criterion):    
    model.eval()
    val_acc = 0.0
    val_loss = 0.0        
    
    with torch.no_grad():        
        end = time.time()        
        for i, (input, target) in enumerate(val_loader):  
            input = input.to(device)
            target = target.to(device)
            output = model(input)            
            val_loss += criterion(output, target).item()
            val_acc += (output.argmax(1) == target).sum().item()                   
        return val_loss/len(val_loader.dataset),val_acc / len(val_loader.dataset)    
    
# 模型预测函数     
def predict(test_loader, model, criterion):    
    model.eval()    
    val_acc = 0.0        
    
    test_pred = []    
    with torch.no_grad():        
        end = time.time()        
        for i, (input, target) in enumerate(test_loader):    
            input = input.to(device)            
            target = target.to(device)            
            output = model(input)            
            test_pred.append(output.data.cpu().numpy())                
            return np.vstack(test_pred)

class LabelSmoothing(nn.Module):
    """NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0):
        """Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        # 此处的self.smoothing即我们的epsilon平滑参数。

    def forward(self, x, target):
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = X_train.append(X_part)
            y_train = y_train.append(y_part)
    train_loader = torch.utils.data.DataLoader(    
        XunFeiDataset(train_df['path'].values, train_df['label'].values,           
        A.Compose([            
            A.RandomCrop(450, 750),
            A.CoarseDropout(p=0.5)
            ])
            ), batch_size=8, shuffle=True, num_workers=0, pin_memory=False)
    
    val_loader = torch.utils.data.DataLoader(    
        XunFeiDataset(train_df['path'].values, train_df['label'].values,            
        A.Compose([            
            A.RandomCrop(450, 750), 
            A.CoarseDropout(p=0.5)
            ])    
            ), batch_size=2, shuffle=False, num_workers=0, pin_memory=False)    
    return train_loader, val_loader

result = pd.DataFrame()
# 定义模型，使用resnet18
print("Creating model----{}".format('resnet34'))

# criterion = nn.CrossEntropyLoss()

lr = 0.0001
n_epochs = 3
k_fold = 5
for i in range(k_fold):
    train_loss_mean = 0.0
    train_acc_mean = 0.0
    val_loss_mean = 0.0
    val_acc_mean = 0.0

    train_loader,val_loader = get_k_fold_data(k_fold, i, train_df['path'], train_df['label'])
    model = timm.create_model('resnet34', pretrained=True, num_classes=24)  # 通过修改模型名字更换不同模型  
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                              max_lr=lr, 
                                              steps_per_epoch=int(len(train_loader)),
                                              epochs=n_epochs,
                                              anneal_strategy='cos')
    print("---------第{}折开始----------".format(i+1))
    epochs = 35
    for epoch in range(epochs):
        print("------epoch{}----------".format(epoch))   
        print("-------Loss----------")      
        train_loss,train_acc = train(train_loader, model, criterion, optimizer) 
        train_loss_mean += train_loss/epochs
        train_acc_mean += train_acc/epochs
        print("loss={}".format(train_loss))  
        print("acc:{}".format(train_acc)) 

        print("-------Val acc----------") 
        val_loss, val_acc = validate(val_loader, model, criterion) 
        val_loss_mean += val_loss/epochs
        val_acc_mean += val_acc/epochs 
        print("loss={}".format(val_loss))  
        print("acc:{}".format(val_acc)) 
    print("第{}折交叉验证train_loss={},train_acc={},val_loss={},val_acc={}".format(i+1,train_loss_mean,train_acc_mean,val_loss_mean,val_acc_mean))
    scheduler.step()
#----------------模型测试----------------
#1. result为每一折测试集预测得到五组种类组合成的Dataframe
#2. pred为每一折测试集得到的原始结果相加
    # 模型预测
    test_loader = torch.utils.data.DataLoader(    
        XunFeiDataset(test_df['path'].values, [0] * test_df.shape[0],            
            A.Compose([            
                A.RandomCrop(450, 750), 
                ])    
                ), batch_size=2, shuffle=False, num_workers=0, pin_memory=False)

    model.eval()    
    val_acc = 0.0        


    test_pred = []    
    with torch.no_grad():  
        for input, _ in tqdm(test_loader):
            # print(img[0])
            input = input.to(device)                      
            output = model(input)            
            test_pred.append(output.data.cpu().numpy())                

    pred = np.vstack(test_pred)

Creating model----resnet34


C:\Users\BiXY\AppData\Local\Temp\ipykernel_21728\3831177988.py:89: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_part)
C:\Users\BiXY\AppData\Local\Temp\ipykernel_21728\3831177988.py:90: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_part)


---------第1折开始----------
------epoch0----------
-------Loss----------
3.2128193378448486
3.4871394634246826
3.2597718238830566
3.1135342121124268
3.291262149810791
3.157421112060547
3.249065399169922
loss=0.4027944167417953
acc:0.04059729351376575
-------Val acc----------
loss=1.588670118690594
acc:0.053663089127391504
------epoch1----------
-------Loss----------
3.192887783050537
3.29885196685791
3.1287553310394287
3.1433603763580322
3.081665515899658
3.1173741817474365
3.1084694862365723
loss=0.3949320823635612
acc:0.06346243583761083
-------Val acc----------
loss=1.5518745856478677
acc:0.09052729818012133
------epoch2----------
-------Loss----------
3.1416585445404053
3.0591979026794434
3.1892499923706055
3.0379488468170166
3.154423713684082
2.9963059425354004
3.109548568725586
loss=0.38752749118635826
acc:0.08306112925804947
-------Val acc----------
loss=1.5165334818132892
acc:0.13905739617358842
------epoch3----------
-------Loss----------
3.042670249938965
3.0039637088775635
2.89

0.5044633150100708
loss=0.04561069490222023
acc:0.9510032664489034
-------Val acc----------
loss=0.08364539245309545
acc:0.9813345776948204
------epoch28----------
-------Loss----------
1.0055181980133057
0.4926382899284363
0.33326879143714905
0.12292416393756866
0.2014501690864563
0.4953933358192444
0.28559887409210205
loss=0.04249355188956396
acc:0.9570695286980868
-------Val acc----------
loss=0.06758987846164136
acc:0.9874008399440037
------epoch29----------
-------Loss----------
0.31538334488868713
0.11974281072616577
0.18956351280212402
0.40988004207611084
0.14714786410331726
0.2538199722766876
0.8177523016929626
loss=0.03728694731227939
acc:0.9640690620625292
-------Val acc----------
loss=0.0603835365688928
acc:0.9874008399440037
------epoch30----------
-------Loss----------
0.2919595241546631
0.17064765095710754
0.2329668551683426
0.2151106894016266
0.10552483052015305
0.16132639348506927
0.09508809447288513
loss=0.033063524858561066
acc:0.9738684087727485
-------Val acc-------

100%|████████████████████████████████████████████████████████████████████████████████| 510/510 [00:24<00:00, 20.95it/s]
C:\Users\BiXY\AppData\Local\Temp\ipykernel_21728\3831177988.py:90: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_part)
C:\Users\BiXY\AppData\Local\Temp\ipykernel_21728\3831177988.py:89: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_part)
C:\Users\BiXY\AppData\Local\Temp\ipykernel_21728\3831177988.py:90: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_part)


---------第2折开始----------
------epoch0----------
-------Loss----------
3.4102354049682617
3.242680072784424
3.272956132888794
3.3755722045898438
2.986555814743042
3.353910446166992
3.0813965797424316
loss=0.4033193574906285
acc:0.046196920205319646
-------Val acc----------
loss=1.5872284109994066
acc:0.06672888474101726
------epoch1----------
-------Loss----------
3.1463685035705566
3.242741823196411
3.12009596824646
3.1986587047576904
3.074615478515625
3.215426445007324
3.0304512977600098
loss=0.3945593642565451
acc:0.07419505366308912
-------Val acc----------
loss=1.5496279647068838
acc:0.10312645823611759
------epoch2----------
-------Loss----------
3.022829294204712
3.0645954608917236
2.9798266887664795
3.098111391067505
3.078979730606079
3.014927625656128
3.3430047035217285
loss=0.3854448413715372
acc:0.11245916938870743
-------Val acc----------
loss=1.5053315368479676
acc:0.15212319178721417
------epoch3----------
-------Loss----------
2.9520039558410645
2.982607841491699
3.061873

In [ ]:
model.eval()    
val_acc = 0.0        


test_pred = []    
with torch.no_grad():  
    for input, _ in tqdm(test_loader):
        # print(img[0])
        input = input.to(device)                      
        output = model(input)            
        test_pred.append(output.data.cpu().numpy())                

pred = np.vstack(test_pred)

In [ ]:

#保存五次预测投票后的结果
label = np.array(result.mode(1)[0],dtype=int)
print(pred.argmax(1))
print([x.split('/')[-1] for x in test_df['path'].values])
# pd.DataFrame(    
#     {        
#         'image': [x.split('/')[-1] for x in test_df['path'].values],        
#         'label': label 
#         }).to_csv('result.csv', index=None)
        
# #保存五次预测的原始结果(1020*24)求和后每一个样本的最大分数对应的类
# pd.DataFrame(    
#     {        
#         'image': [x.split('/')[-1] for x in test_df['path'].values],        
#         'label': pred.argmax(1)
#         }).to_csv('result2.csv', index=None)